# Práctica: Contrastes


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Cargar datos reales: Longley
longley = sm.datasets.longley.load_pandas().data.copy()
longley.head()


### Variables del dataset Longley
- `TOTEMP`: Empleo total (variable dependiente).  
- `GNPDEFL`: Deflactor del GNP.  
- `GNP`: GNP (Producto Nacional Bruto).  
- `UNEMP`: Desempleo (en miles).  
- `ARMED`: Fuerzas Armadas (en miles).  
- `POP`: Población.  
- `YEAR`: Año.


In [ ]:
# Comprobar nombres exactos de columnas en su instalación
longley.columns


## 1) Ajuste del modelo OLS


In [21]:
y = longley["TOTEMP"].astype(float)

X_cols = [c for c in longley.columns if c != "TOTEMP"]
X = longley[X_cols].astype(float)


# Ajustar OLS
model = sm.OLS(y, sm.add_constant(X))  #calcular minimo cuadrados ordinarios
results = model.fit()

print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 TOTEMP   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     330.3
Date:                Tue, 04 Nov 2025   Prob (F-statistic):           4.98e-10
Time:                        08:45:12   Log-Likelihood:                -109.62
No. Observations:                  16   AIC:                             233.2
Df Residuals:                       9   BIC:                             238.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.482e+06    8.9e+05     -3.911      0.0

## 2) Significación **global**

La prueba F en el resumen contrasta:  
$$
H_0: \beta_1=\beta_2=\cdots=\beta_k=0
$$
frente a la alternativa de que **algún** coeficiente es no nulo.


In [22]:
print("Estadístico F global:", results.fvalue)
print("p-valor F global:", results.f_pvalue)

Estadístico F global: 330.28533923414756
p-valor F global: 4.984030528754532e-10


## 3) Significación **individual**

Para cada \( \beta_j \):  
$$
H_0: \beta_j = 0
$$


In [23]:
coef_table = pd.DataFrame({
    "coef": results.params,
    "std_err": results.bse,
    "t": results.tvalues,
    "p>|t|": results.pvalues
})
coef_table


,coef,std_err,t,p>|t|
const,-3.482259e+06,890420.383608,-3.910803,0.003560
GNPDEFL,1.506187e+01,84.914926,0.177376,0.863141
GNP,-3.581918e-02,0.033491,-1.069516,0.312681
UNEMP,-2.020230e+00,0.488400,-4.136427,0.002535
ARMED,-1.033227e+00,0.214274,-4.821985,0.000944
POP,-5.110411e-02,0.226073,-0.226051,0.826212
YEAR,1.829151e+03,455.478499,4.015890,0.003037


## 4) Contrastes sobre **relaciones lineales** \(R\beta = r\)

Además de probar si un coeficiente es 0, a menudo queremos saber si **dos efectos son iguales** o si **una combinación** de efectos cumple cierta relación.


In [24]:
# Ver el orden de parámetros (índices de beta)
results.params.index


Index(['const', 'GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP', 'YEAR'], dtype='object')

### 4.1) Igualdad de dos coeficientes: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}}$

Interpretación: “¿Un incremento marginal en GNP tiene el **mismo** efecto sobre el empleo que un incremento marginal en UNEMP (desempleo)?”


In [25]:

names = results.params.index.tolist()

# Construir restricción: beta_GNP - beta_UNEMP = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNP")]   = 1.0
R[0, names.index("UNEMP")] = -1.0
r = np.array([0.0])

ftest_eq = results.f_test((R, r))   # F-test de igualdad

print("== Igualdad de coeficientes: beta_GNP = beta_UNEMP ==")
print("F-stat:", ftest_eq.fvalue, "p-value:", ftest_eq.pvalue)


== Igualdad de coeficientes: beta_GNP = beta_UNEMP ==
F-stat: 18.86671600551824 p-value: 0.0018676657918953873


### 4.2) Grupo de coeficientes nulos: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}} = 0$


In [26]:
R = np.zeros((2, len(names)))
r = np.zeros(2)

R[0, names.index("GNP")]   = 1.0           # beta_GNP = 0
R[1, names.index("UNEMP")] = 1.0           # beta_UNEMP = 0

ftest_group_zero = results.f_test((R, r))
print("== Grupo cero: beta_GNP = beta_UNEMP = 0 ==")
print("F-stat:", float(ftest_group_zero.fvalue), "p-value:", float(ftest_group_zero.pvalue))


== Grupo cero: beta_GNP = beta_UNEMP = 0 ==
F-stat: 46.7141384089514 p-value: 1.766859000548018e-05


### 4.3) Combinación lineal: $\beta_{\text{GNPDEFL}} + \beta_{\text{POP}} = \beta_{\text{ARMED}}$



In [27]:

# Restricción: beta_GNPDEFL + beta_POP - beta_ARMED = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNPDEFL")] = 1.0
R[0, names.index("POP")]  = 1.0
R[0, names.index("ARMED")]  = -1.0
r = np.array([0.0])

ftest_combo = results.f_test((R, r))
print("== Combinación: beta_GNPDEFL + beta_POP = beta_ARMED ==")
print("F-stat:", float(ftest_combo.fvalue), "p-value:", float(ftest_combo.pvalue))


== Combinación: beta_GNPDEFL + beta_POP = beta_ARMED ==
F-stat: 0.03551135120467506 p-value: 0.8547088211410531


CUANTILES TABLAS

In [28]:
from scipy.stats import t, f, chi2

In [30]:
#para calcular F's experimentales
t.ppf(1-0.025, 16 - 7)

np.float64(2.2621571628540993)

In [32]:
#modelo
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 TOTEMP   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     330.3
Date:                Tue, 04 Nov 2025   Prob (F-statistic):           4.98e-10
Time:                        08:47:18   Log-Likelihood:                -109.62
No. Observations:                  16   AIC:                             233.2
Df Residuals:                       9   BIC:                             238.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.482e+06    8.9e+05     -3.911      0.0

In [ ]:
#comparo: 0,07 no es mayor que 2,26 no puedo rechazar, 4, 126 mayor que ele stadistico experimental

In [33]:
f.ppf(1-0.05, 6, 16-7 )

np.float64(3.373753647039213)

In [35]:
sigma_gorro = results.scale  #margen de cuanto se mueven los errores

In [37]:
#INTERVALO DE CONFIANZA
(16-7)*sigma_gorro/chi2.ppf(1-0.025, 16-7)  #(n-k)*ô^2/(chi2)

np.float64(43969.6296753805)

In [38]:
(16-7)*sigma_gorro/chi2.ppf(0.025, 16-7)

np.float64(309742.00407500827)

I=[43969.6296753805,309742.00407500827]

In [39]:
sigma_gorro  #ô^2 varianza

np.float64(92936.0061674473)

In [40]:
np.sqrt(sigma_gorro) #desviacion típica (como de lejos están los errores de su media )

np.float64(304.8540735621673)